# This notebook aims to visualize the files of created TF records in order to confirm that they have the sought after format

In [1]:
import tensorflow as tf
import meta
%pylab inline

from matplotlib.patches import Rectangle
import numpy as np

from PIL import *
from matplotlib import pyplot as plt
import os
import scipy.io
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# visualization of TFrecord file

In [2]:
data_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/create_tfrecords/data/'

filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                     for i in xrange(0, 9)]# Create a queue that produces the filenames to read.
filename_queue = tf.train.string_input_producer(filenames)

In [ ]:
type(filename_queue)

In [3]:
class SVHNRecord(object):
    pass

result = SVHNRecord()

# Dimensions of the images in the SVHN dataset.
# See http://ufldl.stanford.edu/housenumbers/ for a description of the
# input format.
result.height = 512
result.width = 512
result.depth = 3

reader = tf.TFRecordReader()
result.key, value = reader.read(filename_queue)
value = tf.parse_single_example(
    value,
    # Defaults are not specified since both keys are required.
    features={
        'image_raw': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'label': tf.FixedLenFeature(shape=[], dtype=tf.int64),
        'image_name': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'image_mean':  tf.FixedLenFeature(shape=[], dtype=tf.float32)

    })

# Convert from a string to a vector of uint8 that is record_bytes long.
record_bytes = tf.decode_raw(value['image_raw'], tf.uint8)
# record_bytes.set_shape([32*32*3])
record_bytes = tf.reshape(record_bytes, [result.height, result.width, 3])
print("record bytes::::: ", record_bytes)
# Store our label to result.label and convert to int32
result.name = tf.cast(value['image_name'], tf.string)

result.mean = tf.cast(value['image_mean'], tf.float32)
result.label = tf.cast(value['label'], tf.int32)
result.uint8image = record_bytes
# Image processing for training the network. Note the many random
# distortions applied to the image.
#crop out black pixels
distorted_image = tf.image.central_crop(result.uint8image, 0.9)


('record bytes::::: ', <tf.Tensor 'Reshape:0' shape=(512, 512, 3) dtype=uint8>)


In [4]:
sess = tf.InteractiveSession()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

Below block coutns the file names in the given TFrecord file to ensure that all files in has been written

In [7]:
# print all filenames
names =  []
for i in range(0,10000):
    names.append(sess.run(result.name))

print(set(names))
print(len(set(names)))

set(['33_left', '30_right', '16_left', '10_left', '23_right', '19_left', '17_right', '30_left', '22_left', '15_left', '20_right', '25_right', '21_right', '19_right', '13_right', '15_right', '23_left', '31_right', '25_left', '16_right', '13_left', '22_right', '17_left', '20_left', '21_left', '31_left', '10_right'])
27


In [ ]:
orig_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/orig_images/train'
import os
for count, filename in enumerate(sorted(os.listdir(orig_dir)), start=1):
    #print('Enter {} to select {}'.format(count, filename))

In [ ]:
image_val, label, name, mean = sess.run([distorted_image, result.label, result.name, result.mean])
img = Image.fromarray(image_val, 'RGB')
print(image_val.shape)

print(mean)
#print(image_val)
print(type(image_val))
print(image_val.dtype)
print(np.mean(image_val))
print(np.std(image_val))

print('label is {} and name is {}'.format(label, name))

plt.imshow(img, interpolation='nearest')
plt.figure(figsize=(40,40))
plt.show()

# visualization of .mat file image

In [ ]:
def data_list(data_dir, label_dir):
    '''
    imports: pandas, os, numpy, PIL
    '''
    import pandas as pd
    from PIL import Image
    import numpy as np
    #get labels csv into pandas df
    #below line assumes 
    label_file_name = os.listdir(label_dir)[0]
    label_pd = pd.read_csv(label_dir + label_file_name)
    #initilize container list
    data = [[],[],[]]
    #get filenames om images
    filenames = os.listdir(data_dir)
    #below loop retrieved the 
    for im_name in filenames:
        im = np.asarray(Image.open(data_dir + im_name))
        name = im_name.replace(".jpeg", "")
        label = label_pd.loc[label_pd['image'] == name].iloc[0]['level']
        data[0].append(name)
        data[1].append(im)
        data[2].append(label)
        
    return(data)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
label_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/labels/'
data_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/sample/'
data = data_list(data_dir, label_dir)
plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(data[1]):
    ax = plt.subplot(len(data[1]) / columns + 1, columns, i + 1)
    ax.set_title("Level is {}, name is {}".format(data[2][i], data[0][i]))
    ax.imshow(image)

In [ ]:
label_file_name = os.listdir(label_dir)[0]
label_pd = pd.read_csv(label_dir + label_file_name)
label_pd[3490:3500]

In [ ]:
for i in range(0,10):
    print(label_pd['image'].values[i]+".jpeg")
    print(type(label_pd['image'].values[i]))

In [ ]:
for i in range(0,10):
    print(os.listdir(orig_dir)[i])
    print(type(os.listdir(orig_dir)[i]))

